# Tensorflow models to onnx

Notebook with examples of conversions from tensorflow to onnx models. The tensorflow models were saved using the _save_weights_ (not _save_) function.  
Note: the weights are not provided in the repository and this notebook should be treated as a guide showing how to convert a tensorflow model to onnx.

In [1]:
!pip3 install tf2onnx

You should consider upgrading via the '/home/emilia/alice-code/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
from tensorflow.compat.v1.keras.layers import Input, Dense, LeakyReLU, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.compat.v1.keras.models import Model
from tensorflow.compat.v1.keras.layers import Dense, Reshape, Flatten
from tensorflow.compat.v1.keras.layers import Dropout, BatchNormalization
from scipy.stats import wasserstein_distance

2022-09-28 20:53:58.046782: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-28 20:53:58.046815: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Protons VAE model

The tf2onnx library requires providing path to a saved model, with both architecture and weights. Therefore, it is needed to firstly build a model, load the weights and then save it.

## Define model
The model must have the same structure as the model which weights were saved

In [3]:
latent_dim = 10
cond_dim = 9
poz_dim = 6

In [4]:
x = Input(shape=(latent_dim,))
cond = Input(shape=(cond_dim,))
inputs = Concatenate(axis=1)([x, cond])

g = Dense(7*4*128)(inputs)
g = Reshape((7,4,128))(g)

g = UpSampling2D()(g)
g = Conv2D(128, kernel_size=4, padding='same')(g)
g = BatchNormalization()(g)
g = LeakyReLU(alpha=0)(g)

g = UpSampling2D()(g)
g = Conv2D(64, kernel_size=4, padding='same')(g)
g = BatchNormalization()(g)
g = LeakyReLU(alpha=0)(g)

g = UpSampling2D()(g)
g = Conv2D(32, kernel_size=4, padding='same')(g)
g = BatchNormalization()(g)
g = LeakyReLU(alpha=0)(g)

outputs = Conv2D(1, kernel_size=(1,3) ,activation='relu')(g)

generator = Model([x, cond], outputs, name='generator')
generator.summary()

Model: "generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 9)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 19)           0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 3584)         71680       ['concatenate[0][0]']    

2022-09-28 20:54:03.790269: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-28 20:54:03.790310: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-28 20:54:03.790341: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nbemajerz): /proc/driver/nvidia/version does not exist
2022-09-28 20:54:03.790903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
generator.load_weights('protons_vae/gen_protons_VAE_default_96.h5')

In [6]:
generator.save('generator-protons-vae')

INFO:tensorflow:Assets written to: generator-protons-vae/assets


INFO:tensorflow:Assets written to: generator-protons-vae/assets


### Now it is possible to actually convert the model:

In [7]:
!python -m tf2onnx.convert --saved-model generator-protons-vae --output generator-protons-vae.onnx

2022-09-28 20:55:41.735846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-28 20:55:41.735874: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-09-28 20:55:43.021842: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-28 20:55:43.021865: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-28 20:55:43.021880: 

# ____________________

## Same for gan model

In [6]:
########################### Define Models ############################
latent_dim = 10
cond_dim = 9

############################ generator ############################

x = Input(shape=(latent_dim,))
cond = Input(shape=(cond_dim,))
inputs = Concatenate(axis=1)([x, cond])
layer_1 = Dense(128 * 2)(inputs)
layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)
layer_2 = Dense(128 * 13 * 13)(layer_1_a)
layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)
reshaped = Reshape((13, 13, 128))(layer_2_a)
reshaped_s = UpSampling2D()(reshaped)
conv1 = Conv2D(128, kernel_size=3)(reshaped_s)
conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
conv1_a_s = UpSampling2D()(conv1_a)
conv2 = Conv2D(64, kernel_size=3)(conv1_a_s)
conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)
outputs = Conv2D(1, kernel_size=3, activation='relu')(conv2_a)

generator = Model([x, cond], outputs, name='generator')
generator.summary()

Model: "generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 9)]          0           []                               
                                                                                                  
 concatenate_3 (Concatenate)    (None, 19)           0           ['input_5[0][0]',                
                                                                  'input_6[0][0]']                
                                                                                                  
 dense_8 (Dense)                (None, 256)          5120        ['concatenate_3[0][0]']  

In [7]:
generator.load_weights('gan1_neutrons_default/gen_gan_default_99.h5')

In [8]:
generator.save('generator-gan-default-neutrons')

INFO:tensorflow:Assets written to: generator-gan-default-neutrons/assets


INFO:tensorflow:Assets written to: generator-gan-default-neutrons/assets


In [9]:
!python -m tf2onnx.convert --saved-model generator-gan-default-neutrons --output generator-gan-default-neutrons.onnx

2022-08-30 15:05:13.537688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 15:05:13.537737: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-08-30 15:05:15.494850: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-30 15:05:15.494879: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-30 15:05:15.494899: 